<a id='top'></a><a name='top'></a>
# Chapter 5: Natural Language Generation and Conversion with Transformer

## 5.1 Transformer and Text Generation¶

<table align="left">
  <td>
    <a href="link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

* [Imports and Setup](#setup)
* [5.1 Transformer and Text Generation](#5.1)
    - [5.1.1 What is the Transformer?](#5.1.1)
    - [5.1.2 Text Generation](#5.1.2)

---
<a name='setup'></a><a id='setup'></a>
# Imports and Setup
<a href="#top">[back to top]</a>

In [1]:
import pathlib
from pathlib import Path

data_root = Path("chp05_01")
req_file = data_root / "requirements_5_5_1.txt"

if not data_root.is_dir():
    data_root.mkdir()
else:
    print(f"{data_root} exists.")

chp05_01 exists.


In [2]:
%%writefile {req_file}
datasets==1.11.0
fugashi[unidic]==1.2.1
gensim==3.6.0
japanize_matplotlib==1.1.3
sentencepiece==0.1.97
tqdm==4.64.1
transformers==4.9.0
unidic_lite==1.0.8
watermark==2.3.1

Overwriting chp05_01/requirements_5_5_1.txt


In [5]:
import sys
import os
check1 = ('google.colab' in sys.modules)
check2 = (os.environ.get('CLOUDSDK_CONFIG')=='/content/.config')
IS_COLAB = True if (check1 or check2) else False

if IS_COLAB:
    print("Installing packages")
    !pip install --quiet -r {req_file}
    !python -m unidic download
    !apt-get install jq 2>&1 > /dev/null
    !jq --version
    print("Need to restart runtime after installing sentencepiece.")
    print("> Runtime > Restart runtime ...")
else:
    print("Running locally.")

Installing packages
     |████████████████████████████████| 264 kB 7.2 MB/s 
     |████████████████████████████████| 615 kB 52.7 MB/s 
     |████████████████████████████████| 4.1 MB 43.0 MB/s 
     |████████████████████████████████| 1.3 MB 56.0 MB/s 
     |████████████████████████████████| 2.6 MB 42.9 MB/s 
     |████████████████████████████████| 47.4 MB 1.7 MB/s 
     |████████████████████████████████| 212 kB 47.0 MB/s 
     |████████████████████████████████| 132 kB 59.0 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 880 kB 60.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 1.6 MB 63.1 MB/s 
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:20<00:00, 25.5MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.8/dist-packages/unidic/dicdi

In [2]:
# Standard Library imports
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppress TensorFlog log messages
import pathlib
from pathlib import Path
import shlex
import subprocess
 
# Third-party imports
from collections import Counter
from datasets import load_dataset # HuggingFace Transformers
import json
import logging
import pandas as pd
import sentencepiece as spm
import torch
from tqdm import tqdm
from transformers import T5Tokenizer # This requires sentencepiece
from transformers import AutoModelForCausalLM
from transformers import Trainer
from transformers import TrainingArguments
from watermark import watermark

# suppress logging from transformers
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("transformers.trainer").setLevel(logging.ERROR)
logging.getLogger("datasets").setLevel(logging.ERROR)

_ = torch.manual_seed(42)

def HR():
    print("-"*50)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device:{device}")
HR()

packages_check="datasets,fugashi,gensim,japanize_matplotlib,sentencepiece,tqdm,transformers,unidic_lite,watermark"
print(watermark(packages=packages_check, python=True,machine=True))

device:cpu
--------------------------------------------------
Python implementation: CPython
Python version       : 3.8.15
IPython version      : 7.9.0

datasets           : 1.11.0
fugashi            : 1.2.1
gensim             : 3.6.0
japanize_matplotlib: 1.1.3
sentencepiece      : 0.1.97
tqdm               : 4.64.1
transformers       : 4.9.0
unidic_lite        : 1.0.8
watermark          : 2.3.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.10.133+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [5]:
data_dir = Path("chp05_01")

print(f"data_dir: {data_dir}")

data_dir: chp05_01


---
<a name='5.1'></a><a id='5.1'></a>
# 5.1 Transformer and Text Generation
<a href="#top">[back to top]</a>

<a name='5.1.1'></a><a id='5.1.1'></a>
## 5.1.1 What is the Transformer?
<a href="#top">[back to top]</a>

<a name='5.1.2'></a><a id='5.1.2'></a>
## 5.1.2 Text Generation
<a href="#top">[back to top]</a>

* Use Rinna for our Japanese autoregressive language model GPT-2 model
* Load the pretrained model via `from_pretrained()`.
* To process model input, initialize a corresponding tokenizer.
* Since we are not fine-tuning this model any further, there is no advantage to saving and reusing it.

In [6]:
# Load Rinna — a Japanese GPT-2 model 
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True # due to some bug of tokenizer config loading

model = (AutoModelForCausalLM
        .from_pretrained("rinna/japanese-gpt2-medium")
        .to(device))

type(model)

Downloading:   0%|          | 0.00/806k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [ ]:
inputs = tokenizer(
    "むかしむかし、あるところに、", 
    return_tensors="pt", 
    add_special_tokens=False
    ).to(device)

inputs

{'input_ids': tensor([[    9,   561,  8359,   561,  8359,     7,   382, 12141,     7]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

* Use `model.generate()` to generate the continuation.
* Decode the results and convert back to text with `tokenizer.decode()`

In [ ]:
result = model.generate(
    **inputs,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    max_length=100,
    pad_token_id=2,
    repetition_penalty=1.2
)

print(result.shape)
print(result)

torch.Size([1, 100])
tensor([[    9,   561,  8359,   561,  8359,     7,   382, 12141,     7,   622,
          1391,    10,  4249,   518,   101,    12, 21740,  5363,     8,   235,
          1391,   774,    30,     7,  2134,  5743,   772,  2767,   969,    21,
            49, 22810,    17,   220,  1315,  2874,   341,  7992,    29,     8,
           235,   703, 17309,   543,     9, 11283,   474,  2674,  7192,   124,
           154,     7,   240,  2204, 11139, 13714,  1043,   314,    13,    13,
            13,    21,    20, 16791,    11, 26240, 13391,   842,   969,     8,
         11742,    24,  2604,   108,  7439, 16744,    95,    93,   220,   865,
          2090,    11,   220, 13732, 23120,  5913,    20,  2454,    18,   308,
          1737,    40, 11551,  5363,     8,     9, 16791,    11,     7, 10252]])


In [ ]:
tokenizer.decode(result[0])

'むかしむかし、あるところに、大きな鳥のさえずりが聞こえました。 「鳥さんも、もうすぐ春ですね」というふうにおっしゃっていました。 「あぁ! 春の気配を感じていたら、また冬に戻ってしまいそうだ・・・」とわたしは思ったんですよね。 それから1週間後くらいでしょうか、「お母様はお元気ですか?」と電話をいただいたことがありました。 わたしは、あの'

In [30]:
# Another iteration, using this text from weathernews.jp:
# 雨や雪の降りやすい天気が続きます。降り方が強まり、雷やアラレを伴うおそれもあります。
# 寒い日が多く、特に朝晩はグッと冷え込みます。体調管理を万全に。
# https://weathernews.jp/onebox/tenki/niigata/15461/

inputs = tokenizer(
    "雨や雪の降りやすい天気が続きます。降り方が強まり、雷やアラレを伴うおそれもあります。寒い日が多く、特に朝晩はグッと冷え込みます。体調管理を万全に。", 
    return_tensors="pt", 
    add_special_tokens=False
    ).to(device)

result = model.generate(
    **inputs,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    max_length=100,
    pad_token_id=2,
    repetition_penalty=1.2
)

tokenizer.decode(result[0])

'雨や雪の降りやすい天気が続きます。降り方が強まり、雷やアラレを伴うおそれもあります。寒い日が多く、特に朝晩はグッと冷え込みます。体調管理を万全に。 【9月27日(木)】秋の全国交通安全運動が実施されます!【10月5日まで実施中】 [pdfファイル/1.17mb] 10月1日から9日にかけて、全国の警察署で一斉取締りを実施しています。'

In [31]:
# Text from Twitter:
# 12/8(木) 15:20現在本日16時からのTeNYテレビ新潟さんの夕方のニュースは、
# かぐらメインゲレンデからの生中継でお届け？？ドラ〜✨#naeba #かぐら #夕方ワイド新潟一番
# https://twitter.com/Naebanow/status/1600738428211335170

inputs = tokenizer(
    "12/8(木) 15:20現在本日16時からのTeNYテレビ新潟さんの夕方のニュースは、かぐらメインゲレンデからの生中継でお届け？？ドラ〜✨#naeba #かぐら #夕方ワイド新潟一番", 
    return_tensors="pt", 
    add_special_tokens=False
    ).to(device)

result = model.generate(
    **inputs,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    max_length=100,
    pad_token_id=2,
    repetition_penalty=1.2
)

tokenizer.decode(result[0])

'12/8(木) 15:20現在本日16時からのtenyテレビ新潟さんの夕方のニュースは、かぐらメインゲレンデからの生中継でお届け??ドラ〜<unk> #naeba #かぐら #夕方ワイド新潟一番 pic.twitter.com/dm1nxb9wcz — @nippon_kanazawa (@kanazawayamagensen) 2017年10月12日'